In [101]:
# pip3 install jupyter
# pyarrow, fastparquet
# pip3 install pandas

import pandas as pd
converters = {
    'lpep_pickup_datetime': lambda x : pd.to_datetime(x, format='%Y%m%d %H:%M:%S'),
    'Lpep_dropoff_datetime': lambda x : pd.to_datetime(x, format='%Y%m%d %H:%M:%S'),
    'Store_and_fwd_flag': lambda x : False if x == None else x == 'Y',
    'Ehail_fee': lambda x : 0 if not x else x,
    'Trip_type ': lambda x : -1 if not x else int(x)
}
df = pd.read_csv('green_tripdata_2013-09.csv', skiprows=[1,1], converters=converters).rename(columns=lambda x: x.strip().lower())
df.to_parquet('/tmp/green_tripdata_2013-09.parquet')
print("Converted!")

Converted!
